# अध्याय 7: चैट एप्लिकेशन बनाना
## Github Models API त्वरित शुरुआत

यह नोटबुक [Azure OpenAI सैंपल्स रिपॉजिटरी](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst) से ली गई है, जिसमें वे नोटबुक्स शामिल हैं जो [Azure OpenAI](notebook-azure-openai.ipynb) सेवाओं का उपयोग करती हैं।


# अवलोकन  
"बड़े भाषा मॉडल वे फंक्शन होते हैं जो टेक्स्ट को टेक्स्ट में बदलते हैं। जब कोई इनपुट टेक्स्ट दिया जाता है, तो एक बड़ा भाषा मॉडल यह अनुमान लगाने की कोशिश करता है कि अगला टेक्स्ट क्या होगा"(1)। यह "क्विकस्टार्ट" नोटबुक उपयोगकर्ताओं को उच्च-स्तरीय LLM अवधारणाओं, AML के साथ शुरुआत करने के लिए आवश्यक मुख्य पैकेज, प्रॉम्प्ट डिज़ाइन का एक सरल परिचय, और विभिन्न उपयोग मामलों के कुछ छोटे उदाहरणों से परिचित कराएगी।


## विषय सूची  

[सामान्य जानकारी](../../../../07-building-chat-applications/python)  
[OpenAI सेवा का उपयोग कैसे करें](../../../../07-building-chat-applications/python)  
[1. अपनी OpenAI सेवा बनाना](../../../../07-building-chat-applications/python)  
[2. इंस्टॉलेशन](../../../../07-building-chat-applications/python)    
[3. क्रेडेंशियल्स](../../../../07-building-chat-applications/python)  

[उपयोग के मामले](../../../../07-building-chat-applications/python)    
[1. टेक्स्ट का सारांश बनाना](../../../../07-building-chat-applications/python)  
[2. टेक्स्ट को वर्गीकृत करना](../../../../07-building-chat-applications/python)  
[3. नए प्रोडक्ट नाम जनरेट करना](../../../../07-building-chat-applications/python)  
[4. क्लासिफायर को फाइन-ट्यून करना](../../../../07-building-chat-applications/python)  

[संदर्भ](../../../../07-building-chat-applications/python)


### अपना पहला प्रॉम्प्ट बनाएं  
यह छोटा सा अभ्यास आपको Github Models में किसी मॉडल को प्रॉम्प्ट सबमिट करने की बुनियादी जानकारी देगा, एक आसान टास्क "सारांश" के लिए।

**कदम**:  
1. अगर आपने अब तक नहीं किया है, तो अपने python environment में `azure-ai-inference` लाइब्रेरी इंस्टॉल करें।  
2. स्टैंडर्ड हेल्पर लाइब्रेरी लोड करें और Github Models के लिए क्रेडेंशियल सेटअप करें।  
3. अपने टास्क के लिए एक मॉडल चुनें  
4. मॉडल के लिए एक साधारण प्रॉम्प्ट बनाएं  
5. अपनी रिक्वेस्ट मॉडल API को सबमिट करें!


### 1. `azure-ai-inference` स्थापित करें


In [ ]:
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

### 3. सही मॉडल चुनना  
GPT-3.5-turbo या GPT-4 मॉडल प्राकृतिक भाषा को समझ सकते हैं और जनरेट कर सकते हैं।


In [ ]:
# Select the General Purpose curie model for text
model_name = "gpt-4o"

## 4. प्रॉम्प्ट डिज़ाइन  

"बड़े भाषा मॉडल्स का जादू यह है कि जब इन्हें बहुत बड़ी मात्रा में टेक्स्ट पर इस प्रेडिक्शन एरर को कम करने के लिए ट्रेन किया जाता है, तो ये मॉडल्स उन कॉन्सेप्ट्स को सीख लेते हैं जो इन प्रेडिक्शन्स के लिए उपयोगी होते हैं। उदाहरण के लिए, वे ये कॉन्सेप्ट्स सीखते हैं"(1):

* कैसे शब्दों की स्पेलिंग की जाती है
* व्याकरण कैसे काम करता है
* पैराफ्रेज़ कैसे करना है
* सवालों के जवाब कैसे देना है
* बातचीत कैसे करना है
* कई भाषाओं में कैसे लिखना है
* कोडिंग कैसे करनी है
* आदि

#### बड़े भाषा मॉडल को कैसे नियंत्रित करें  
"बड़े भाषा मॉडल के सभी इनपुट्स में सबसे ज्यादा प्रभावशाली टेक्स्ट प्रॉम्प्ट होता है(1)।

बड़े भाषा मॉडल्स को आउटपुट देने के लिए कुछ तरीकों से प्रॉम्प्ट किया जा सकता है:

निर्देश: मॉडल को बताएं कि आपको क्या चाहिए  
पूरक: मॉडल को उस चीज़ की शुरुआत पूरी करने के लिए प्रेरित करें जो आप चाहते हैं  
डेमोंस्ट्रेशन: मॉडल को दिखाएं कि आपको क्या चाहिए, या तो:  
प्रॉम्प्ट में कुछ उदाहरण देकर  
फाइन-ट्यूनिंग ट्रेनिंग डाटासेट में सैकड़ों या हजारों उदाहरण देकर"

#### प्रॉम्प्ट बनाने के तीन मूलभूत नियम हैं:

**दिखाएं और बताएं**। यह स्पष्ट करें कि आपको क्या चाहिए, या तो निर्देशों के माध्यम से, उदाहरणों के माध्यम से, या दोनों के संयोजन से। अगर आप चाहते हैं कि मॉडल किसी लिस्ट को वर्णमाला के क्रम में रैंक करे या किसी पैराग्राफ को सेंटिमेंट के आधार पर वर्गीकृत करे, तो उसे दिखाएं कि आप यही चाहते हैं।

**गुणवत्ता वाला डाटा दें**। अगर आप क्लासिफायर बनाना चाहते हैं या मॉडल से कोई पैटर्न फॉलो करवाना चाहते हैं, तो सुनिश्चित करें कि पर्याप्त उदाहरण हों। अपने उदाहरणों को प्रूफरीड जरूर करें — मॉडल आमतौर पर इतनी समझ रखता है कि बेसिक स्पेलिंग मिस्टेक्स को पहचान सके और आपको जवाब दे सके, लेकिन वह यह भी मान सकता है कि यह जानबूझकर किया गया है और इससे जवाब प्रभावित हो सकता है।

**अपनी सेटिंग्स जांचें।** टेम्परेचर और top_p सेटिंग्स यह नियंत्रित करती हैं कि मॉडल जवाब जनरेट करते समय कितना निश्चित या विविध होगा। अगर आप उससे ऐसा जवाब मांग रहे हैं जिसमें केवल एक ही सही उत्तर है, तो इन्हें कम रखें। अगर आप ज्यादा विविध जवाब चाहते हैं, तो इन्हें ज्यादा रखें। इन सेटिंग्स के साथ लोग जो सबसे बड़ी गलती करते हैं, वह यह मानना है कि ये "चतुराई" या "क्रिएटिविटी" को नियंत्रित करती हैं।

स्रोत: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## टेक्स्ट का सारांश  
#### चुनौती  
टेक्स्ट का सारांश निकालें और टेक्स्ट के अंत में 'tl;dr:' जोड़ें। ध्यान दें कि मॉडल बिना किसी अतिरिक्त निर्देश के कई तरह के कार्य करना समझता है। आप tl;dr से भी ज्यादा वर्णनात्मक प्रॉम्प्ट्स का प्रयोग करके मॉडल के व्यवहार को बदल सकते हैं और अपनी पसंद के अनुसार सारांश प्राप्त कर सकते हैं(3)।  

हाल ही के शोध में यह दिखाया गया है कि बड़े टेक्स्ट कॉर्पस पर प्री-ट्रेनिंग और फिर किसी खास टास्क पर फाइन-ट्यूनिंग करने से कई NLP टास्क और बेंचमार्क्स पर काफी अच्छे परिणाम मिले हैं। हालांकि यह तरीका आमतौर पर आर्किटेक्चर में टास्क-अज्ञेय होता है, फिर भी इसमें हजारों या दसियों हजार टास्क-विशिष्ट फाइन-ट्यूनिंग डेटा की जरूरत होती है। इसके विपरीत, इंसान आमतौर पर कुछ उदाहरणों या साधारण निर्देशों से ही नई भाषा से जुड़ा कार्य कर सकते हैं - जो कि मौजूदा NLP सिस्टम्स के लिए अब भी काफी मुश्किल है। यहां हम दिखाते हैं कि भाषा मॉडल्स का स्केल बढ़ाने से टास्क-अज्ञेय, फ्यू-शॉट परफॉर्मेंस में काफी सुधार होता है, और कई बार यह पिछले स्टेट-ऑफ-द-आर्ट फाइन-ट्यूनिंग तरीकों के बराबर भी पहुंच जाता है।



Tl;dr


# कई उपयोग मामलों के लिए अभ्यास  
1. पाठ का सारांश बनाएं  
2. पाठ को वर्गीकृत करें  
3. नए उत्पाद नाम बनाएं


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## टेक्स्ट को वर्गीकृत करें  
#### चुनौती  
आइटम्स को उन श्रेणियों में वर्गीकृत करें जो इनफरेंस के समय दी जाती हैं। नीचे दिए गए उदाहरण में, हम प्रॉम्प्ट में श्रेणियां और वर्गीकृत करने के लिए टेक्स्ट दोनों प्रदान करते हैं (*playground_reference)।

ग्राहक पूछताछ: नमस्ते, मेरे लैपटॉप कीबोर्ड की एक कुंजी हाल ही में टूट गई है और मुझे उसका रिप्लेसमेंट चाहिए:

वर्गीकृत श्रेणी:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## नए प्रोडक्ट नाम बनाएं
#### चुनौती
उदाहरण शब्दों से प्रोडक्ट के नाम बनाएं। यहां हम उस प्रोडक्ट के बारे में जानकारी देते हैं जिसके लिए नाम बनाने हैं। साथ ही, एक मिलता-जुलता उदाहरण भी दिया गया है ताकि आपको वांछित पैटर्न समझ में आ सके। हमने temperature वैल्यू भी ज्यादा रखी है ताकि जवाब ज्यादा रैंडम और इनोवेटिव आएं।

प्रोडक्ट विवरण: एक होम मिल्कशेक मेकर
सीड शब्द: तेज, हेल्दी, कॉम्पैक्ट।
प्रोडक्ट नाम: होमशेकर, फिट शेकर, क्विकशेक, शेक मेकर

प्रोडक्ट विवरण: ऐसे जूते जो किसी भी पैर के साइज में फिट हो सकते हैं।
सीड शब्द: एडैप्टेबल, फिट, ऑम्नी-फिट।


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# संदर्भ  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio Examples](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [GPT-3 को टेक्स्ट वर्गीकृत करने के लिए फाइन-ट्यून करने के सर्वोत्तम तरीके](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# अधिक सहायता के लिए  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)


# योगदानकर्ता
* [Chew-Yean Yam](https://www.linkedin.com/in/cyyam/)



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। यद्यपि हम सटीकता के लिए प्रयासरत हैं, कृपया ध्यान दें कि स्वचालित अनुवादों में त्रुटियाँ या गलतियाँ हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को ही प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
